# Ζητούμενο 5

### Να υλοποιηθεί το Query 5 χρησιμοποιώντας το DataFrame ή SQL API. Να εκτελέσετε την υλοποίησή σας χρησιμοποιώντας συνολικούς πόρους 8 cores και 16GB μνήμης με τα παρακάτω configurations:
> * 2 executors × 4 cores/8GB memory
> * 4 executors × 2 cores/4GB memory
> * 8 executors × 1 core/2 GB memory

### * Conf 1

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,
175,application_1738075734771_0176,pyspark,idle,Link,Link,None,
183,application_1738075734771_0184,pyspark,idle,Link,Link,None,
186,application_1738075734771_0187,pyspark,idle,Link,Link,None,
193,application_1738075734771_0194,pyspark,idle,Link,Link,None,
195,application_1738075734771_0196,pyspark,idle,Link,Link,None,
215,application_1738075734771_0216,pyspark,idle,Link,Link,None,


In [2]:
# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
245,application_1738075734771_0246,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 2
Executor Memory: 8g
Executor Cores: 4

In [3]:
# DF query 5 execution 2*(4/8)
from pyspark.sql import functions as F
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import time

group_number = "5"
base_s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/CrimeData/"

parquet_s3_path = base_s3_path + "parquet/"

spark = SparkSession \
    .builder \
    .appName("DF query 5 execution 2*(4/8)") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

start_time = time.time()

la_police_stations_schema = StructType([StructField("X", DoubleType(), False),
    StructField("Y", DoubleType(), False),
    StructField("FID", IntegerType(), False),
    StructField("DIVISION", StringType(), False),
    StructField("LOCATION", StringType(), False),
    StructField("PREC", IntegerType(), False)])

la_police_stations_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(la_police_stations_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv")
    
la_police_stations_df = la_police_stations_df.withColumn("station_point", ST_Point("X", "Y"))

#la_police_stations_df.printSchema()
#la_police_stations_df.show(5)

unified_crime_data_df = spark.read.parquet(parquet_s3_path)
null_island_filtered = unified_crime_data_df.filter(~((col("lat") == 0) & (col("lon") == 0))) \
    .withColumn("crime_point", ST_Point("lon", "lat"))

#null_island_filtered.show(5)
#null_island_filtered.printSchema()

unified_df = null_island_filtered.crossJoin(la_police_stations_df)
unified_df = unified_df.select(
    "dr_no",
    "station_point",  # from la_police_stations_df
    "crime_point",    # from null_island_filtered
    "DIVISION"        # from la_police_stations_df
)
#unified_df.show(5)

unified_df = unified_df.withColumn("distance_km", ST_DistanceSphere("station_point", "crime_point")/1000)
#unified_df.show(22)

unified_df_min_distance = unified_df \
    .withColumn("row_num", F.row_number().over(Window.partitionBy("dr_no").orderBy("distance_km"))) \
    .filter(F.col("row_num") == 1) \
    .drop("row_num")

#unified_df_min_distance.show(5)
#unified_df_min_distance.filter(F.col("dr_no") == 1307355).show(5)

division_summary_df = unified_df_min_distance \
    .groupBy("DIVISION") \
    .agg(
        F.avg("distance_km").alias("average_distance"),
        F.count("dr_no").alias("#")
    ) \
    .orderBy(F.col("#").desc())  # Sort by the count column in descending order

# Show the resulting DataFrame
division_summary_df.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|DIVISION        |average_distance  |#     |
+----------------+------------------+------+
|HOLLYWOOD       |2.0762639601787183|224340|
|VAN NUYS        |2.9533697428197803|210134|
|SOUTHWEST       |2.1913988057808833|188901|
|WILSHIRE        |2.592665532978764 |185996|
|77TH STREET     |1.7165449719700954|171827|
|OLYMPIC         |1.723603697178095 |170897|
|NORTH HOLLYWOOD |2.643006094141564 |167854|
|PACIFIC         |3.850070655307917 |161359|
|CENTRAL         |0.9924764374568906|153871|
|RAMPART         |1.5345341879190124|152736|
|SOUTHEAST       |2.4218662158881807|152176|
|WEST VALLEY     |3.03567121631407  |138643|
|TOPANGA         |3.2969548417555536|138217|
|FOOTHILL        |4.250921708424982 |134896|
|HARBOR          |3.702561599356522 |126747|
|HOLLENBECK      |2.680181237706819 |115837|
|WEST LOS ANGELES|2.7924572890341226|115781|
|NEWTON          |1.6346357397097429|111110|
|NORTHEAST       |3.6236655246040876|108109|
|MISSION  

### * Conf 2

In [4]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
246,application_1738075734771_0247,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,
175,application_1738075734771_0176,pyspark,idle,Link,Link,None,
183,application_1738075734771_0184,pyspark,idle,Link,Link,None,
186,application_1738075734771_0187,pyspark,idle,Link,Link,None,
193,application_1738075734771_0194,pyspark,idle,Link,Link,None,
195,application_1738075734771_0196,pyspark,idle,Link,Link,None,
215,application_1738075734771_0216,pyspark,idle,Link,Link,None,


In [5]:
# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 4
Executor Memory: 4g
Executor Cores: 2

In [6]:
# DF query 5 execution 4*(2/4)
from pyspark.sql import functions as F
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import time

group_number = "5"
base_s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/CrimeData/"

parquet_s3_path = base_s3_path + "parquet/"

spark = SparkSession \
    .builder \
    .appName("DF query 5 execution 4*(2/4)") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

start_time = time.time()

la_police_stations_schema = StructType([StructField("X", DoubleType(), False),
    StructField("Y", DoubleType(), False),
    StructField("FID", IntegerType(), False),
    StructField("DIVISION", StringType(), False),
    StructField("LOCATION", StringType(), False),
    StructField("PREC", IntegerType(), False)])

la_police_stations_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(la_police_stations_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv")
    
la_police_stations_df = la_police_stations_df.withColumn("station_point", ST_Point("X", "Y"))

#la_police_stations_df.printSchema()
#la_police_stations_df.show(5)

unified_crime_data_df = spark.read.parquet(parquet_s3_path)
null_island_filtered = unified_crime_data_df.filter(~((col("lat") == 0) & (col("lon") == 0))) \
    .withColumn("crime_point", ST_Point("lon", "lat"))

#null_island_filtered.show(5)
#null_island_filtered.printSchema()

unified_df = null_island_filtered.crossJoin(la_police_stations_df)
unified_df = unified_df.select(
    "dr_no",
    "station_point",  # from la_police_stations_df
    "crime_point",    # from null_island_filtered
    "DIVISION"        # from la_police_stations_df
)
#unified_df.show(5)

unified_df = unified_df.withColumn("distance_km", ST_DistanceSphere("station_point", "crime_point")/1000)
#unified_df.show(22)

unified_df_min_distance = unified_df \
    .withColumn("row_num", F.row_number().over(Window.partitionBy("dr_no").orderBy("distance_km"))) \
    .filter(F.col("row_num") == 1) \
    .drop("row_num")

#unified_df_min_distance.show(5)
#unified_df_min_distance.filter(F.col("dr_no") == 1307355).show(5)

division_summary_df = unified_df_min_distance \
    .groupBy("DIVISION") \
    .agg(
        F.avg("distance_km").alias("average_distance"),
        F.count("dr_no").alias("#")
    ) \
    .orderBy(F.col("#").desc())  # Sort by the count column in descending order

# Show the resulting DataFrame
division_summary_df.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|DIVISION        |average_distance  |#     |
+----------------+------------------+------+
|HOLLYWOOD       |2.0762639601787183|224340|
|VAN NUYS        |2.9533697428197803|210134|
|SOUTHWEST       |2.1913988057808833|188901|
|WILSHIRE        |2.592665532978764 |185996|
|77TH STREET     |1.7165449719700954|171827|
|OLYMPIC         |1.723603697178095 |170897|
|NORTH HOLLYWOOD |2.643006094141564 |167854|
|PACIFIC         |3.850070655307917 |161359|
|CENTRAL         |0.9924764374568906|153871|
|RAMPART         |1.5345341879190124|152736|
|SOUTHEAST       |2.4218662158881807|152176|
|WEST VALLEY     |3.03567121631407  |138643|
|TOPANGA         |3.2969548417555536|138217|
|FOOTHILL        |4.250921708424982 |134896|
|HARBOR          |3.702561599356522 |126747|
|HOLLENBECK      |2.680181237706819 |115837|
|WEST LOS ANGELES|2.7924572890341226|115781|
|NEWTON          |1.6346357397097429|111110|
|NORTHEAST       |3.6236655246040876|108109|
|MISSION  

### * Conf 3

In [7]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
247,application_1738075734771_0248,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,
175,application_1738075734771_0176,pyspark,idle,Link,Link,None,
183,application_1738075734771_0184,pyspark,idle,Link,Link,None,
186,application_1738075734771_0187,pyspark,idle,Link,Link,None,
193,application_1738075734771_0194,pyspark,idle,Link,Link,None,
195,application_1738075734771_0196,pyspark,idle,Link,Link,None,
215,application_1738075734771_0216,pyspark,idle,Link,Link,None,


In [8]:
# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 8
Executor Memory: 2g
Executor Cores: 1

In [9]:
# DF query 5 execution 8*(1/2)
from pyspark.sql import functions as F
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import time

group_number = "5"
base_s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/CrimeData/"

parquet_s3_path = base_s3_path + "parquet/"

spark = SparkSession \
    .builder \
    .appName("DF query 5 execution 8*(1/2)") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

start_time = time.time()

la_police_stations_schema = StructType([StructField("X", DoubleType(), False),
    StructField("Y", DoubleType(), False),
    StructField("FID", IntegerType(), False),
    StructField("DIVISION", StringType(), False),
    StructField("LOCATION", StringType(), False),
    StructField("PREC", IntegerType(), False)])

la_police_stations_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(la_police_stations_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv")
    
la_police_stations_df = la_police_stations_df.withColumn("station_point", ST_Point("X", "Y"))

#la_police_stations_df.printSchema()
#la_police_stations_df.show(5)

unified_crime_data_df = spark.read.parquet(parquet_s3_path)
null_island_filtered = unified_crime_data_df.filter(~((col("lat") == 0) & (col("lon") == 0))) \
    .withColumn("crime_point", ST_Point("lon", "lat"))

#null_island_filtered.show(5)
#null_island_filtered.printSchema()

unified_df = null_island_filtered.crossJoin(la_police_stations_df)
unified_df = unified_df.select(
    "dr_no",
    "station_point",  # from la_police_stations_df
    "crime_point",    # from null_island_filtered
    "DIVISION"        # from la_police_stations_df
)
#unified_df.show(5)

unified_df = unified_df.withColumn("distance_km", ST_DistanceSphere("station_point", "crime_point")/1000)
#unified_df.show(22)

unified_df_min_distance = unified_df \
    .withColumn("row_num", F.row_number().over(Window.partitionBy("dr_no").orderBy("distance_km"))) \
    .filter(F.col("row_num") == 1) \
    .drop("row_num")

#unified_df_min_distance.show(5)
#unified_df_min_distance.filter(F.col("dr_no") == 1307355).show(5)

division_summary_df = unified_df_min_distance \
    .groupBy("DIVISION") \
    .agg(
        F.avg("distance_km").alias("average_distance"),
        F.count("dr_no").alias("#")
    ) \
    .orderBy(F.col("#").desc())  # Sort by the count column in descending order

# Show the resulting DataFrame
division_summary_df.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|DIVISION        |average_distance  |#     |
+----------------+------------------+------+
|HOLLYWOOD       |2.0762639601787183|224340|
|VAN NUYS        |2.9533697428197803|210134|
|SOUTHWEST       |2.1913988057808833|188901|
|WILSHIRE        |2.592665532978764 |185996|
|77TH STREET     |1.7165449719700954|171827|
|OLYMPIC         |1.723603697178095 |170897|
|NORTH HOLLYWOOD |2.643006094141564 |167854|
|PACIFIC         |3.850070655307917 |161359|
|CENTRAL         |0.9924764374568906|153871|
|RAMPART         |1.5345341879190124|152736|
|SOUTHEAST       |2.4218662158881807|152176|
|WEST VALLEY     |3.03567121631407  |138643|
|TOPANGA         |3.2969548417555536|138217|
|FOOTHILL        |4.250921708424982 |134896|
|HARBOR          |3.702561599356522 |126747|
|HOLLENBECK      |2.680181237706819 |115837|
|WEST LOS ANGELES|2.7924572890341226|115781|
|NEWTON          |1.6346357397097429|111110|
|NORTHEAST       |3.6236655246040876|108109|
|MISSION  